In [2]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb


In [3]:
import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import bert_squad_model
from checklist.pred_wrapper import PredictorWrapper
model = bert_squad_model.BertSquad()
invert = lambda a: model.predict_pairs([(x[1], x[0]) for x in a])
new_pp = PredictorWrapper.wrap_predict(invert)

In [3]:
model.predict_pairs([('Who is smarter?', 'John is smart')])

['John']

In [4]:
editor = checklist.editor.Editor()
editor.tg

In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [7]:
def format_squad(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'Q: %s\n' % (q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [8]:
import json
def load_squad(fold='validation'):
    answers = []
    data = []
    ids = []
    files = {
        'validation': '/home/marcotcr/datasets/squad/dev-v1.1.json',
        'train': '/home/marcotcr//datasets/squad/train-v1.1.json',
        }
    f = json.load(open(files[fold]))
    for t in f['data']:
        for p in t['paragraphs']:
            context = p['context']
            for qa in p['qas']:
                data.append({'passage': context, 'question': qa['question'], 'id': qa['id']})
                answers.append(set([(x['text'], x['answer_start']) for x in qa['answers']]))
    return data, answers


In [9]:
import pickle
data, answers =  load_squad()
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_squad.pkl', 'rb'))
pairs = [(x['passage'], x['question']) for x in data]
processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in pairs]

In [10]:
suite = TestSuite()

## Vocabulary

In [11]:
print(', '.join(editor.suggest('{first_name} is {mask} than {first_name2}.')[:60]))

better, older, smarter, worse, younger, taller, different, more, stronger, bigger, less, shorter, tougher, other, greater, wiser, larger, smaller, faster, richer, cooler, darker, nicer, higher, weaker, happier, longer, hotter, closer, lower, harder, safer, heavier, slower, stranger, easier, quicker, deeper, brighter, simpler, colder, healthier, wealthier, thicker, thinner, cleaner, lighter, quieter, cheaper, poorer, louder, newer, warmer, sharper, wider, lesser, superior, further, earlier, clearer


In [12]:
adj = ['old', 'smart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]


In [13]:
adj[2]

('tall', 'tall')

In [14]:
t = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is less {adj[1]}?'
    ),(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is {adj[0]}er?'
    )
    ],
    labels = ['{first_name1}','{first_name}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=500,
    save=True
    )
name = 'A is COMP than B. Who is more / less COMP?'
description = ''
test = MFT(**t, name=name, description=description, capability='Vocabulary')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 988 examples



Test cases:      494
Fails (rate):    99 (20.0%)

Example fails:
C: Jonathan is older than Olivia.
Q: Who is less old?
A: Olivia
P: Jonathan


----
C: Samantha is slower than Jordan.
Q: Who is less slow?
A: Jordan
P: Samantha is slower than Jordan


----
C: Kelly is weaker than Ashley.
Q: Who is less weak?
A: Ashley
P: Kelly


----


In [15]:
def crossproduct(t):
    # takes the output of editor.template and does the cross product of contexts and qas
    ret = []
    ret_labels = []
    for x in t.data:
        cs = x['contexts']
        qas = x['qas']
        d = list(itertools.product(cs, qas))
        ret.append([(x[0], x[1][0]) for x in d])
        ret_labels.append([x[1][1] for x in d])
    t.data = ret
    t.labels = ret_labels
    return t


In [16]:
state = editor.suggest('John is very {mask} about the project.')[:20]
print(', '.join(editor.suggest('John is {mask} {state} about the project.', state=state)[:30]))
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

very, pretty, extremely, quite, also, still, more, really, not, fairly, incredibly, rather, now, generally, already, clearly, relatively, highly, particularly, so, surprisingly, most, currently, certainly, super, definitely, increasingly, being, especially, understandably


In [17]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ), 
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ), 
            
        ]
        
    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?'
desc = ''
test = MFT(**t, name=name, description=desc, capability='Vocabulary')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)


Predicting 5964 examples



Test cases:      497
Fails (rate):    454 (91.3%)

Example fails:
C: John is happy about the project. Maria is very happy about the project.
Q: Who is least happy about the project?
A: John
P: Maria


----
C: Mark is vocal about the project. Danielle is super vocal about the project.
Q: Who is most vocal about the project?
A: Danielle
P: Mark

C: Mark is vocal about the project. Danielle is super vocal about the project.
Q: Who is least vocal about the project?
A: Mark
P: Danielle


----
C: Justin is excited about the project. Brian is super excited about the project.
Q: Who is most excited about the project?
A: Brian
P: Justin

C: Justin is excited about the project. Brian is super excited about the project.
Q: Who is least excited about the project?
A: Justin
P: Brian


----


## Taxonomy

### Size, chape, color, age, material

In [18]:
import munch
order = ['size', 'shape', 'age', 'color']
props = []
properties = {
    'color' : ['red', 'blue','yellow', 'green', 'pink', 'white', 'black', 'orange', 'grey', 'purple', 'brown'],
    'size' : ['big', 'small', 'tiny', 'enormous'],
    'age' : ['old', 'new'],
    'shape' : ['round', 'oval', 'square', 'triangular'],
    'material' : ['iron', 'wooden', 'ceramic', 'glass', 'stone']
}
for i in range(len(order)):
    for j in range(i + 1, len(order)):
        p1, p2 = order[i], order[j]
        for v1, v2 in itertools.product(properties[p1], properties[p2]):
            props.append(munch.Munch({
                'p1': p1,
                'p2': p2,
                'v1': v1,
                'v2': v2,
            }))


In [19]:
print(', '.join(editor.suggest('There is {a:p.v1} {p.v2} {mask} in the room.', p=props, verbose=False)[:30]))
objects = ['box', 'clock', 'table', 'object', 'toy', 'painting', 'sculpture', 'thing', 'figure']


couch, sofa, wall, carpet, chair, table, light, door, clock, lamp, mirror, bed, TV, bar, window, tree, box, desk, painting, fridge, curtain, screen, fan, camera, frame, wallpaper, rug, cabinet, elephant, television


In [20]:
t = crossproduct(editor.template(
    {
        'contexts': [
            'There is {a:p.v1} {p.v2} {obj} in the room.',
            'There is {a:obj} in the room. The {obj} is {p.v1} and {p.v2}.',
        ],
        'qas': [
            (
                'What {p.p1} is the {obj}?',
                '{p.v1}'
            ), 
            (
                'What {p.p2} is the {obj}?',
                '{p.v2}'
            ), 
            
        ]
        
    },
    obj=objects,
    p=props,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'size, shape, age, color'
desc = ''
test = MFT(**t, name=name, description=desc, capability='Taxonomy')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 2000 examples



Test cases:      500
Fails (rate):    412 (82.4%)

Example fails:
C: There is a box in the room. The box is enormous and round.
Q: What size is the box?
A: enormous
P: enormous and round

C: There is a box in the room. The box is enormous and round.
Q: What shape is the box?
A: round
P: enormous and round


----
C: There is a big yellow clock in the room.
Q: What size is the clock?
A: big
P: big yellow

C: There is a clock in the room. The clock is big and yellow.
Q: What size is the clock?
A: big
P: big and yellow


----
C: There is a painting in the room. The painting is round and black.
Q: What shape is the painting?
A: round
P: round and black


----


### Professions vs nationalities

In [21]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')

In [22]:
def clean(string):
    return string.lstrip('[a,the,an,in,at] ').rstrip('.')

In [23]:
def expect_squad(x, pred, conf, label=None, meta=None):
    return clean(pred) == clean(label)
expect_squad = Expect.single(expect_squad)

In [24]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ), 
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ), 
            
        ]
        
    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True,
    ))
name = 'Profession vs nationality'
test = MFT(**t, name=name, expect=expect_squad, description='',  capability='Taxonomy')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 5000 examples



Test cases:      500
Fails (rate):    247 (49.4%)

Example fails:
C: Ashley is a Japanese adviser.
Q: What is Ashley's job?
A: adviser
P: Japanese adviser


----
C: Matthew is a Japanese educator.
Q: What is Matthew's job?
A: educator
P: Japanese educator


----
C: Timothy is an Indian analyst.
Q: What is Timothy's job?
A: analyst
P: Indian analyst


----


### Animal vs vehicle

In [25]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ), 
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Animal vs Vehicle'
test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)


Predicting 2000 examples



Test cases:      500
Fails (rate):    128 (25.6%)

Example fails:
C: Austin has a cow and a tractor.
Q: What vehicle does Austin have?
A: tractor
P: a cow and a tractor

C: Austin has a tractor and a cow.
Q: What animal does Austin have?
A: cow
P: a tractor and a cow


----
C: Alexander has a lizard and a firetruck.
Q: What animal does Alexander have?
A: lizard
P: a lizard and a firetruck

C: Alexander has a firetruck and a lizard.
Q: What animal does Alexander have?
A: lizard
P: a firetruck and a lizard


----
C: Angela has a bull and a tractor.
Q: What animal does Angela have?
A: bull
P: a bull and a tractor

C: Angela has a bull and a tractor.
Q: What vehicle does Angela have?
A: tractor
P: a bull and a tractor

C: Angela has a tractor and a bull.
Q: What animal does Angela have?
A: bull
P: a tractor and a bull


----


In [26]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} bought {a:animal}. {first_name2} bought {a:vehicle}.',
            '{first_name2} bought {a:vehicle}. {first_name} bought {a:animal}.',
        ],
        'qas': [
            (
                'Who bought an animal?',
                '{first_name}'
            ), 
            (
                'Who bought a vehicle?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Animal vs Vehicle v2'
test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)

Predicting 1984 examples



Test cases:      496
Fails (rate):    130 (26.2%)

Example fails:
C: David bought a train. Kayla bought a guinea pig.
Q: Who bought an animal?
A: Kayla
P: David


----
C: David bought a bull. Mary bought a train.
Q: Who bought a vehicle?
A: Mary
P: David

C: Mary bought a train. David bought a bull.
Q: Who bought a vehicle?
A: Mary
P: Mary bought a train. David


----
C: Christina bought a tractor. Mary bought a cow.
Q: Who bought an animal?
A: Mary
P: Christina


----


In [27]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'),
            ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'),
            ('humble', 'modest'), ('courageous', 'brave'), ('happy', 'joyful'), ('scared', 'frightened'),
           ]

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[0]}. {first_name2} is very {s2[0]}.',
            '{first_name2} is very {s2[0]}. {first_name} is very {s1[0]}.',
        ],
        'qas': [
            (
                'Who is {s1[1]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[1]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=250,
    save=True
   ))
t += crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is very {s1[1]}. {first_name2} is very {s2[1]}.',
            '{first_name2} is very {s2[1]}. {first_name} is very {s1[1]}.',
        ],
        'qas': [
            (
                'Who is {s1[0]}?',
                '{first_name}'
            ), 
            (
                'Who is {s2[0]}?',
                '{first_name2}'
            ), 
            
        ]
        
    },
    s=synonyms,
    remove_duplicates=True,
    nsamples=250,
    save=True
    )) 
name = 'Synonyms'
test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1788 examples



Test cases:      447
Fails (rate):    0 (0.0%)


In [28]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs))#list(set(comp_pairs + [(x[1], x[0]) for x in comp_pairs]))

In [29]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {comp[0]} than {first_name1}.',
            '{first_name1} is {comp[1]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is {comp[1]}?',
                '{first_name1}',
            ),
            (
                'Who is {comp[0]}?',
                '{first_name}',
            )
            
        ]
        ,
    },
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'A is COMP than B. Who is antonym(COMP)? B'
test = MFT(**t, name=name, description='', capability='Taxonomy')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1984 examples



Test cases:      496
Fails (rate):    334 (67.3%)

Example fails:
C: Jacob is colder than Matthew.
Q: Who is warmer?
A: Matthew
P: Jacob


----
C: Ryan is older than Amber.
Q: Who is younger?
A: Amber
P: Ryan


----
C: Andrea is quieter than Maria.
Q: Who is louder?
A: Maria
P: Andrea


----


In [30]:
antonym_adjs = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is more {a[0]} than {first_name1}.',
            '{first_name1} is more {a[1]} than {first_name}.',
            '{first_name} is less {a[1]} than {first_name1}.',
            '{first_name1} is less {a[0]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is more {a[0]}?',
                '{first_name}',
            ),
            (
                'Who is less {a[0]}?',
                '{first_name1}',
            ),
            (
                'Who is more {a[1]}?',
                '{first_name1}',
            ),
            (
                'Who is less {a[1]}?',
                '{first_name}',
            ),
        ]
        ,
    },
    a = antonym_adjs,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'A is more X than B. Who is more antonym(X)? B. Who is less X? B. Who is more X? A. Who is less antonym(X)? A.'
test = MFT(**t, name=name, description='', capability='Taxonomy')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 7856 examples



Test cases:      491
Fails (rate):    491 (100.0%)

Example fails:
C: Heather is more unhappy than Rachel.
Q: Who is more happy?
A: Rachel
P: Heather

C: Rachel is more happy than Heather.
Q: Who is more unhappy?
A: Heather
P: Rachel

C: Rachel is more happy than Heather.
Q: Who is less unhappy?
A: Rachel
P: Heather


----
C: Michelle is more religious than Jordan.
Q: Who is more secular?
A: Jordan
P: Michelle

C: Michelle is more religious than Jordan.
Q: Who is less secular?
A: Michelle
P: Jordan

C: Jordan is more secular than Michelle.
Q: Who is more religious?
A: Michelle
P: Jordan


----
C: Elizabeth is more cautious than Timothy.
Q: Who is more brave?
A: Timothy
P: Elizabeth

C: Elizabeth is more cautious than Timothy.
Q: Who is less brave?
A: Elizabeth
P: Timothy

C: Timothy is more brave than Elizabeth.
Q: Who is more cautious?
A: Elizabeth
P: Timothy


----


## Robustness

typos

In [31]:
def question_typo(x):
    return (x[0], Perturb.add_typos(x[1]))
t = Perturb.perturb(pairs, question_typo, nsamples=500)
test = INV(**t, name='Question typo', capability='Robustness', description='')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad)
suite.add(test, overwrite=True)

Predicting 1000 examples



Test cases:      500
Fails (rate):    58 (11.6%)

Example fails:
Q: What is the theory that this King's name is the origin of "Huguenot" called?
P: Hugues hypothesis

Q: What is the theory that this King's name is the origin of "Huguenot" calle?d
P: The "Hugues hypothesis


----
Q: Who did the Broncos beat to win their division in 2015?
P: Pittsburgh Steelers

Q: Who did the Broncos beat to win theird ivision in 2015?
P: New England Patriots


----
Q: How did Luther describe his learning at the university?
P: rote learning

Q: How did Luther describe his leanring at the university?
P: a beerhouse and whorehouse


----


Contractions

In [32]:
def contractions(x):
    conts = Perturb.contractions(x[1])
    return [(x[0], a) for a in conts]
t = Perturb.perturb(pairs, contractions, nsamples=500)
test = INV(**t, name='Question contractions', capability='Robustness', description='')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad)
suite.add(test)

Predicting 1005 examples



Test cases:      500
Fails (rate):    17 (3.4%)

Example fails:
Q: What is a connection identifier 
P: packets include a connection identifier rather than address information

Q: What's a connection identifier 
P: The packets include a connection identifier rather than address information


----
Q: Where did the Meuse flow before the flood? 
P: just south of today's line Merwede-Oude Maas

Q: Where'd the Meuse flow before the flood? 
P: just south of today's line Merwede-Oude Maas to the North Sea


----
Q: What do red algal chloroplasts have that green chloroplasts don't?
P: phycobilisomes

Q: What do red algal chloroplasts have that green chloroplasts do not?
P: chlorophyll b


----


Add random sentence

In [33]:
random_sentences = set()
for x, _ in processed_pairs:
    for y in x.sents:
        random_sentences.add(y.text)
random_sentences = list(random_sentences)

In [34]:
# len(random_sentences)

In [35]:
def add_random_sentence(x, **kwargs):
    random_s = np.random.choice(random_sentences)
    while random_s in x[0]:
        random_s = np.random.choice(random_sentences)
    random_s = random_s.strip('.') + '. '
    meta = ['add to end: %s' % random_s, 'add to beg: %s' % random_s]
    return [(x[0] + random_s, x[1]), (random_s + x[0], x[1])], meta

def format_add(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s\n' % meta
    return ret

t = Perturb.perturb(pairs, add_random_sentence, nsamples=500, meta=True)
test = INV(**t, name='Add random sentence to context', capability='Robustness', description='')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_add)
suite.add(test)

Predicting 1500 examples



Test cases:      500
Fails (rate):    49 (9.8%)

Example fails:
Q: What is the main reason consulting pharmacists are increasingly working directly with patients?
P: many elderly people are now taking numerous medications

Q: What is the main reason consulting pharmacists are increasingly working directly with patients?
P: many elderly people are now taking numerous medications but continue to live outside of institutional settings
Perturb: add to beg: Kublai botched his campaigns against Annam, Champa, and Java, but won a Pyrrhic victory against Burma. 


----
Q: The adaptive immune system must distinguish between what types of molecules?
P: self and non-self molecules

Q: The adaptive immune system must distinguish between what types of molecules?
P: self and non-self
Perturb: add to beg: Many churches preserve sculptured fonts, capitals, and more importantly mosaics, which were common in Norman Italy and drew heavily on the Greek heritage. 


----
Q: What did these flights test on 

## NER

In [36]:
import re
def change_thing(change_fn):
    def change_both(cq, **kwargs):
        context, question = cq
        a = change_fn(context, meta=True)
        if not a:
            return None
        changed, meta = a
        ret = []
        for c, m in zip(changed, meta):
            new_q = re.sub(r'\b%s\b' % re.escape(m[0]), m[1], question.text)
            ret.append((c, new_q))
        return ret, meta
    return change_both
            

In [97]:
def expect_same(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    if not meta:
        return pred == orig_pred
    return pred == re.sub(r'\b%s\b' % re.escape(meta[0]), meta[1], orig_pred)

def format_replace(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

def format_replace_context(x, pred, conf, label=None, meta=None):
    ret = format_squad_with_context(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

In [39]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_names), nsamples=500, meta=True)

test = INV(**t, name='Change name everywhere', capability='NER',
          description='', expect=Expect.pairwise(expect_same))
test.run(new_pp)
test.summary(3, format_example_fn=format_replace)
suite.add(test, overwrite=True)

Predicting 5500 examples



Test cases:      500
Fails (rate):    29 (5.8%)

Example fails:
Q: What American actor is also a university graduate?
P: Philip Kaufman

Q: What American actor is also a university graduate?
P: Ed Asner
Perturb: Carl Van Vechten -> David Rivera

Q: What American actor is also a university graduate?
P: Ed Asner
Perturb: Carl Van Vechten -> Daniel Campbell


----
Q: The Codex Forster is a collection of notebooks by which famous Italian Renaissance polymath?
P: Leonardo da Vinci

Q: The Codex Forster is a collection of notebooks by which famous Italian Renaissance polymath?
P: Michael Myers
Perturb: Leonardo da Vinci's -> Michael Myers

Q: The Codex Forster is a collection of notebooks by which famous Italian Renaissance polymath?
P: Christopher Miller
Perturb: Leonardo da Vinci's -> Christopher Miller


----
Q: Who did Luther view to be the Antichrist?
P: the papacy, and the Roman Church

Q: Who did Adam view to be the Antichrist?
P: the papacy
Perturb: Luther -> Adam


----


In [113]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_location), nsamples=500, meta=True)

test = INV(**t, name='Change location everywhere', capability='NER',
          description='', expect=Expect.pairwise(expect_same))
test.run(new_pp)
test.summary(3, format_example_fn=format_replace)
suite.add(test, overwrite=True)

Predicting 5500 examples



Test cases:      500
Fails (rate):    37 (7.4%)

Example fails:
Q: Who was sacked as the first half clock expired?
P: Newton

Q: Who was sacked as the first half clock expired?
P: DeMarcus Ware
Perturb: Newton -> Buffalo

Q: Who was sacked as the first half clock expired?
P: DeMarcus Ware
Perturb: Newton -> Minneapolis


----
Q: What is an example of an article of uniform clothing typically present in Australian private schools?
P: compulsory blazer

Q: What is an example of an article of uniform clothing typically present in Australian private schools?
P: a compulsory blazer
Perturb: Australia -> Peru


----
Q: Ludwig Krapf recorded the name was what?
P: both Kenia and Kegnia

Q: Ludwig Krapf recorded the name was what?
P: Kenia and Kegnia
Perturb: Kenya -> Philippines

Q: Ludwig Krapf recorded the name was what?
P: Kenia and Kegnia
Perturb: Kenya -> Morocco


----


## Temporal

In [41]:
t = crossproduct(editor.template(
    {
        'contexts': [
            'Both {first_name} and {first_name2} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
            'Both {first_name2} and {first_name} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
        ],
        'qas': [
            (
                'Who is {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'There was a change in profession'
test = MFT(**t, expect=expect_squad, capability='Temporal', name=name, description='' )
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 964 examples



Test cases:      482
Fails (rate):    200 (41.5%)

Example fails:
C: Both Aaron and Mark were photographers, but there was a change in Aaron, who is now a producer.
Q: Who is a producer?
A: Aaron
P: Aaron and Mark were photographers, but there was a change in Aaron


----
C: Both Brian and David were educators, but there was a change in David, who is now an artist.
Q: Who is an artist?
A: David
P: David were educators, but there was a change in David


----
C: Both Megan and Aaron were artists, but there was a change in Aaron, who is now an architect.
Q: Who is an architect?
A: Aaron
P: Aaron were artists, but there was a change in Aaron


----


In [42]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} became a {prof} before {first_name2} did.',
            '{first_name2} became a {prof} after {first_name} did.',
        ],
        'qas': [
            (
                'Who became a {prof} first?',
                '{first_name}'
            ), 
            (
                'Who became a {prof} last?',
                '{first_name2}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Understanding before / after -> first / last.'
test = MFT(**t, expect=expect_squad, capability='Temporal', name=name, description='' )
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)


Predicting 1984 examples



Test cases:      496
Fails (rate):    411 (82.9%)

Example fails:
C: Amber became a economist before Daniel did.
Q: Who became a economist last?
A: Daniel
P: Amber


----
C: Emily became a educator after Amy did.
Q: Who became a educator first?
A: Amy
P: Emily

C: Emily became a educator after Amy did.
Q: Who became a educator last?
A: Emily
P: Amy


----
C: Samuel became a academic after Jacob did.
Q: Who became a academic first?
A: Jacob
P: Samuel


----


## Negation

In context

In [43]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is not {a:prof}. {first_name2} is.',
            '{first_name2} is {a:prof}. {first_name} is not.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Negation in context, may or may not be in question'
test = MFT(**t, expect=expect_squad, capability='Negation', name=name, description='' )
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1996 examples



Test cases:      499
Fails (rate):    337 (67.5%)

Example fails:
C: Jason is an intern. Emily is not.
Q: Who is not an intern?
A: Emily
P: Jason


----
C: Brandon is not an author. Robert is.
Q: Who is an author?
A: Robert
P: Brandon


----
C: William is not an architect. Ryan is.
Q: Who is an architect?
A: Ryan
P: William


----


Not in context:

In [44]:

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:prof}. {first_name2} is {a:prof2}.',
            '{first_name2} is {a:prof2}. {first_name} is {a:prof}.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is {a:prof2}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Negation in question only.'
test = MFT(**t, expect=expect_squad, capability='Negation', name=name, description='' )
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3848 examples



Test cases:      481
Fails (rate):    481 (100.0%)

Example fails:
C: Daniel is an actor. Kevin is an analyst.
Q: Who is not an actor?
A: Kevin
P: Daniel

C: Daniel is an actor. Kevin is an analyst.
Q: Who is not an analyst?
A: Daniel
P: Kevin

C: Kevin is an analyst. Daniel is an actor.
Q: Who is not an actor?
A: Kevin
P: Daniel


----
C: Amanda is an organizer. Stephanie is an activist.
Q: Who is not an organizer?
A: Stephanie
P: Amanda

C: Amanda is an organizer. Stephanie is an activist.
Q: Who is not an activist?
A: Amanda
P: Stephanie

C: Stephanie is an activist. Amanda is an organizer.
Q: Who is not an organizer?
A: Stephanie
P: Amanda


----
C: Jacob is an accountant. Elizabeth is a nurse.
Q: Who is not an accountant?
A: Elizabeth
P: Jacob

C: Jacob is an accountant. Elizabeth is a nurse.
Q: Who is not a nurse?
A: Jacob
P: Elizabeth

C: Elizabeth is a nurse. Jacob is an accountant.
Q: Who is not an accountant?
A: Elizabeth
P: Jacob


----


## Fairness spinoff

In [45]:
import collections
fewer_profs = ['doctor', 'nurse', 'secretary', 'CEO']
t = editor.template(
    [
        ('{male} is not {a:prof}, {female} is.', 'Who is {a:prof}?', '{female}', 'woman', '{prof}'),
        ('{female} is not {a:prof}, {male} is.', 'Who is {a:prof}?', '{male}', 'man', '{prof}'),
    ],
#     prof=professions + ['doctor'],
    prof=fewer_profs,
    remove_duplicates=True,
    nsamples=1000,
    unroll=True,
    save=True,
    )
data = [(d[0], d[1]) for d in t.data]
labels = [d[2] for d in t.data]
meta = [(d[3], d[4]) for d in t.data]

test = MFT(data, expect=expect_squad, labels=labels, meta=meta, templates=t.templates,
          name='M/F failure rates should be similar for different professions', capability='Fairness',
          description='Using negation in context.')
test.run(new_pp)

def print_fair(test):
    c = collections.Counter(test.meta)
    fail = collections.Counter([tuple(x) for x in np.array(test.meta)[test.fail_idxs()]])
    profs = set()
    for sex, prof in fail:
        profs.add(prof)
    prof_fail = {}
    get_fail = lambda f:fail[f] / c[f]
    for prof in profs:
        fail_m = get_fail(('man', prof))
        fail_f = get_fail(('woman', prof))
        prof_fail[prof] = (fail_m, fail_f)
    print('%-13s fail_men fail_women (count)' % 'profession')
    for prof, vs in sorted(prof_fail.items(), key=lambda x:max(x[1][0], x[1][1]), reverse=True):
        fail_m, fail_f = vs
        print('%-13s   %.1f      %.1f     (%d)' % (prof, 100 * fail_m, 100 * fail_f, c[('man', prof)]))
print_fair(test)
suite.add(test)

Predicting 2000 examples



profession    fail_men fail_women (count)
CEO             0.17      0.97     (267)
doctor          0.03      0.89     (247)
secretary       0.60      0.04     (253)
nurse           0.58      0.41     (233)


## Coref

Basic coref

In [46]:
if 'actress' in professions:
    professions.remove('actress')

In [47]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{female} and {male} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{male} and {female} are friends. She is {a:prof2}, and he is {a:prof1}.',
            '{female} and {male} are friends. She is {a:prof2}, and he is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{male}'
            ), 
            (
                'Who is {a:prof2}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Basic coref, he / she'
test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 3816 examples



Test cases:      477
Fails (rate):    477 (100.0%)

Example fails:
C: Katherine and Paul are friends. He is an advisor, and she is an analyst.
Q: Who is an analyst?
A: Katherine
P: Paul

C: Paul and Katherine are friends. She is an analyst, and he is an advisor.
Q: Who is an advisor?
A: Paul
P: Katherine

C: Katherine and Paul are friends. She is an analyst, and he is an advisor.
Q: Who is an analyst?
A: Katherine
P: Paul


----
C: Paul and Taylor are friends. He is an organizer, and she is a photographer.
Q: Who is an organizer?
A: Paul
P: Paul and Taylor

C: Taylor and Paul are friends. He is an organizer, and she is a photographer.
Q: Who is a photographer?
A: Taylor
P: Paul

C: Paul and Taylor are friends. She is a photographer, and he is an organizer.
Q: Who is an organizer?
A: Paul
P: Taylor


----
C: Rebecca and Gabriel are friends. He is an executive, and she is an author.
Q: Who is an author?
A: Rebecca
P: Gabriel

C: Gabriel and Rebecca are friends. She is an author, and he 

In [48]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. His mom is {a:prof}.',
            '{female} and {male} are friends. His mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{male}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=250,
    ))
t += crossproduct(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. Her mom is {a:prof}.',
            '{female} and {male} are friends. Her mom is {a:prof}.',
        ],
        'qas': [
            (
                'Whose mom is {a:prof}?',
                '{female}'
            ), 
        ]
        
    },
    save=True,
    prof=professions,
    remove_duplicates=True,
    nsamples=250,
    ))

name = 'Basic coref, his / her'
test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1000 examples



Test cases:      500
Fails (rate):    459 (91.8%)

Example fails:
C: Erica and Paul are friends. Her mom is an academic.
Q: Whose mom is an academic?
A: Erica
P: Paul


----
C: Leah and Chad are friends. Her mom is an advisor.
Q: Whose mom is an advisor?
A: Leah
P: Chad


----
C: Megan and Nathan are friends. Her mom is an accountant.
Q: Whose mom is an accountant?
A: Megan
P: Megan and Nathan


----


Former, latter

In [49]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} and {first_name2} are friends. The former is {a:prof1}.',
            '{first_name2} and {first_name} are friends. The latter is {a:prof1}.',
            '{first_name} and {first_name2} are friends. The former is {a:prof1} and the latter is {a:prof2}.',
            '{first_name2} and {first_name} are friends. The former is {a:prof2} and the latter is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True
    ))
name = 'Former / Latter'
test = MFT(**t, expect=expect_squad, name=name, description='', capability='Coref')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1900 examples



Test cases:      475
Fails (rate):    475 (100.0%)

Example fails:
C: Jason and Mark are friends. The former is an interpreter.
Q: Who is an interpreter?
A: Jason
P: Mark

C: Jason and Mark are friends. The former is an interpreter and the latter is an investigator.
Q: Who is an interpreter?
A: Jason
P: Mark


----
C: Lauren and Andrew are friends. The former is an educator.
Q: Who is an educator?
A: Lauren
P: Andrew

C: Lauren and Andrew are friends. The former is an educator and the latter is an economist.
Q: Who is an educator?
A: Lauren
P: Andrew


----
C: Jonathan and William are friends. The former is a producer.
Q: Who is a producer?
A: Jonathan
P: William

C: Jonathan and William are friends. The former is a producer and the latter is an interpreter.
Q: Who is a producer?
A: Jonathan
P: Jonathan and William


----


## SRL

In [50]:
import pattern
import pattern.en
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('loves')[0]
b = pattern.en.tenses('stolen')[0]
pverb = [(pattern.en.conjugate(v, *a), pattern.en.conjugate(v, *b)) for v in pverb]

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} {v[0]} {first_name2}.',
            '{first_name2} is {v[1]} by {first_name}.',
        ],
        'qas': [
            (
                'Who {v[0]}?',
                '{first_name}'
            ), 
            (
                'Who is {v[1]}?',
                '{first_name2}'
            ), 
        ]
        
    },
    v=pverb,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Agent / object distinction'
test = MFT(**t, expect=expect_squad, name=name, description='', capability='SRL')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)

Predicting 1988 examples



Test cases:      497
Fails (rate):    302 (60.8%)

Example fails:
C: Ethan attacks Alyssa.
Q: Who is attacked?
A: Alyssa
P: Ethan


----
C: Lisa deserves Megan.
Q: Who is deserved?
A: Megan
P: Lisa

C: Megan is deserved by Lisa.
Q: Who deserves?
A: Lisa
P: Megan


----
C: Jennifer accepts Michelle.
Q: Who is accepted?
A: Michelle
P: Jennifer accepts Michelle


----


In [51]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} {v[0]} {first_name2}. {first_name2} {v[0]} {first_name3}.',
            '{first_name} {v[0]} {first_name2}. {first_name3} is {v[1]} by {first_name2}.',
            '{first_name2} is {v[1]} by {first_name}. {first_name2} {v[0]} {first_name3}.',
            '{first_name2} is {v[1]} by {first_name}. {first_name3} is {v[1]} by {first_name2}.',
        ],
        'qas': [
            (
                'Who {v[0]} {first_name2}?',
                '{first_name}'
            ), 
            (
                'Who {v[0]} {first_name3}?',
                '{first_name2}'
            ), 
            (
                'Who is {v[1]} by {first_name}?',
                '{first_name2}'
            ), 
            (
                'Who is {v[1]} by {first_name2}?',
                '{first_name3}'
            ), 
        ]
        
    },
    save=True,
    v=pverb,
    remove_duplicates=True,
    nsamples=500,
    ))
name = 'Agent / object distinction with 3 agents'
test = MFT(**t, expect=expect_squad, name=name, description='', capability='SRL')
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test)


Predicting 7728 examples



Test cases:      483
Fails (rate):    462 (95.7%)

Example fails:
C: Jonathan is attacked by Jason. Jonathan attacks Joshua.
Q: Who is attacked by Jonathan?
A: Joshua
P: Jason


----
C: Anna is followed by Sarah. Anna follows Amanda.
Q: Who follows Amanda?
A: Anna
P: Sarah. Anna

C: Anna is followed by Sarah. Anna follows Amanda.
Q: Who is followed by Anna?
A: Amanda
P: Sarah

C: Anna is followed by Sarah. Amanda is followed by Anna.
Q: Who is followed by Anna?
A: Amanda
P: Sarah. Amanda


----
C: Emma prefers Alexander. Noah is preferred by Alexander.
Q: Who prefers Noah?
A: Alexander
P: Emma

C: Emma prefers Alexander. Noah is preferred by Alexander.
Q: Who is preferred by Emma?
A: Alexander
P: Noah

C: Alexander is preferred by Emma. Alexander prefers Noah.
Q: Who is preferred by Alexander?
A: Noah
P: Emma


----


In [52]:
suite.save('/home/marcotcr/tmp/squad_suite.pkl')

In [57]:
suite.summary(n=3, format_example_fn=format_squad_with_context)

Vocabulary

A is COMP than B. Who is more / less COMP?
Test cases:      494
Fails (rate):    99 (20.0%)

Example fails:
C: Shannon is darker than Amy.
Q: Who is less dark?
A: Amy
P: Shannon


----
C: Alexander is shorter than Jeffrey.
Q: Who is shorter?
A: Alexander
P: Jeffrey


----
C: Jeremy is shorter than Sarah.
Q: Who is less short?
A: Sarah
P: Jeremy


----


Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?
Test cases:      497
Fails (rate):    454 (91.3%)

Example fails:
C: Sean is super cautious about the project. Jordan is cautious about the project.
Q: Who is most cautious about the project?
A: Sean
P: Jordan

C: Jordan is cautious about the project. Sean is super cautious about the project.
Q: Who is most cautious about the project?
A: Sean
P: Jordan

C: Jordan is cautious about the project. Sean is super cautious about the project.
Q: Who is least cautious about the project?
A: Jordan
P: Sean


----
C: Tiffany is ambitious about the project. Steven

In [5]:
suite = TestSuite.from_file('/home/marcotcr/tmp/squad_suite.pkl')

In [7]:
print_fair(suite.tests['M/F failure rates should be similar for different professions'])

profession    fail_men fail_women (count)
CEO             16.9      96.6     (267)
doctor          3.2      89.1     (247)
secretary       60.5      4.0     (253)
nurse           57.9      41.2     (233)
